# Docker como herramienta de desarrollo

## Usando Docker para desarrollar aplicaciones

Ahora que tenemos el conocimiento de los contenedores y construir imagenes. Vemos como Docker puede ayudarnos en el dia a dia para construir applicaciones.

Clonaremos un repo: https://github.com/platzi/docker

    git clone https://github.com/platzi/docker

Hay varios archivos, pero es una aplicacion en Node JS, y exploramos el *index.js*. No necesitamos ser expertos en dicho lenguaje pero ayuda. 

![](https://i.imgur.com/7O0TGKY.png)

Se expone una api en la ruta base, y va recibir una peticio, y va intentar conectarse a una instancia de Mongo DB. Si puede, dice que lo pudo hacer, y si no arroja un error.

### Examinando el Docker File

![](https://i.imgur.com/mTjX7GN.png)

- Basado en node: 12
- Con la instrucion copi, se copian todos los archivos...

Es importante entender lo que significa el punto(.). Le estoy diciendo donde estoy parado, y donde estoy parado esta hablando del *contexto de build*, 

- A continuacion se especifica un directorio de trabajo, que es equivalente a hacer: *cd usr/src*. Dentro de la imagen obviamente

- Ya dentro de ese directorio, se ejecuta *npm install*. Que instala las dependencias relacionadas en el *package.json*.

- Luego, expone un puerto: 3000. Es decir, se pueda acceder al contenedor a traves del puerto.

- POr ultimo, CMD, donde se define el comando o proceso por defecto que va a ejecutarse, cuando se corra un contendedo basado en esta imagen

### Construyendo la imagen

Asegurarnos estemos en la carpeta /docker que acabamos de clonar.

    docker build -t platziapp .

Tardara un poco...

Y se puede verificar ya esta disponible en mi lista de imagenes:

![](https://i.imgur.com/YkuzKKf.png)

### Corriendo un Contenedor 

    docker run --rm -p 3000:3000 platziapp 

- con la opcion --rm le indica a docker una vez el contenedor se detenga, borrarlo automaticamente. 
- -p 3000:3000   *(PuertoMaquinaHost:PuertoDelContenedor)*

Vamos al navegador, en mi maquina local en el puerto 

![](https://i.imgur.com/wbe8A7n.png)





Obviamente, nos da un error porque la aplicacion no esta vinculada a ningun MOngo, como era de esperarse.

Recomendacion, correr el contenedor con la opcion detach:

    docker run --detach  --rm --publish 3200:3000 platziapp

Ahora lo veremos en el localhost:3200

## Aprovechando el caché de capas para estructurar correctamente tus imágenes

Una de las herramientas que nos de DOCKER para agilizar el tiempo de trabajo es el cache de capas.

Si estamos intentando construir una capa o layer que ya existe. Docker se da cuenta y no la construye. 

Una prueba de esto, es si intentamos construir la imagen del modulo anterior, no tomara nada de tiempo.

Sin embargo hay varias consideraciones. Por ejemplo se quiere cambiar la version de node, digamos a 14. Y como esta es la capa base, va invalidar todos los layers de ahi en adelante. 

    FROM node:14

Pero no es el unico problema. Si por ejemplo se quiere actualizar el codigo, pues vuelve a empezar, y tendra que copiar todo el codigo de nuestro folder:

    COPY [".", "/usr/src/"]

¿Y entonces porque si solo cambie el codigo, tiene entonces que volver a ejecutar por ejemplo:

    RUN npm install?

Esto no es lo mas conveniente. 😱

Seguramente habra que reestructurar ese *Docker file* porque esta mal estructurado. 

### Empezando a reestructurar el Dockerfile

Como ves *npm install* necesita algunos archivos que se llaman *package.json*, y no necesita nada mas dependencias. Asi que los copiamos de primeras:

    COPY ["package.json", "package-lock.json ", "/usr/src/"]

COPY acepta *n* argumentos, en este caso dos que se copiaran al directorio */usr/src*, y a continuacion se corre:

    RUN npm install

Asi estamos indicando que solo se ejecute *npm install* si cambia una dependencia. Y a continuacion si copiamos los archivos como lo habiamos hecho antes.

    COPY [".", "/usr/src/"]

¿Pero esta opcion no te copiara de vuelta *package.json* y *package-lock.json*? 😏

NO. POrque *DOCKER* es "inteligente" y se da cuenta que si la operacion anterior significa sobreescribir esos dos archivos, se fija si hay diferencia entre lo que se le esta pidiendo copie, y lo que esta. Como no hay diferencia entre lo que se copio anteriormente, *DOCKER* se ahorra ese paso y no los copia. Pero si traera todos los demas.

Haremos un pequeño cambio en el *index.js*, y corramos un nuevo *build*:

![](https://i.imgur.com/wfH2rKr.png)

    docker build -t platziapp .

Igualmente tardara un poco, no tanto como la primera vez, y finalmente construira la imagen.



#### ¿Que pasa si cambio algo en el codigo?

Realizamos un cambio cosmetico en *index.js* y volvemos a buildiar con el comando anterior. Pues esta vez no se demoro nada 😇. Ya estamos usando el *cache* para mejorar el proceso de construcion de imagenes.

Como ejercicio, corramos un nuevo contenedor con la imagen recien creada

    docker run --detach --rm --publish 3200:3000 platziapp

Esto es muy util, porque en este caso teniamos solo dos dependencias, pero en una aplicacion real podrian haber decenas incluso cientos de dependencias.

#### ¿Sera posible no tener que buildiar cada vez que se cambia algo en el codigo?

Esto seria muy practico. De tal forma que el contenedor se actualize cada vez que cambiamos algo de codigo, pero sin rebuildiar todo. 

Compartiremos al container el codigo mediante un *bind mount*, pero antes usaremos un proceso propio de *node* para que cambie algun archivo, se actualize o se reinicie la aplicacion con el nuevo codigo. 

Dentro del package *json* hay una herramienta llamada *nodemon* que hace precisamente eso. Para ello cambiaremos nuestro CMD o *Command*:

    CMD ["npx", "nodemon", "index.js"]

Y volvemos a buildiar, lo caul ocurre muy rapido.

    docker build -t platziapp .

Correremos un container agregandole un *bind mount*. Lo Haremos por pasos para mostrarte un trucazo

    docker run -d --rm -p 3200:3000 --v

Como ves hace falta colocar toda la ruta que es larguisima, asi que enves de eso escribimos el comando pwd entre unas comillas especiales y le damos TAB:

    👉docker run -d --rm -p 3200:3000 -v `pwd` 

Y a continuacion me autocompletara:

    👉docker run -d --rm -p 3200:3000 -v /home/carlos/Documents/curso_docker/docker

Y colocamos todo los demas, para correrlo sin la opcion --detatch

    docker run --rm -p 3200:3000 -v /home/carlos/Documents/curso_docker/docker/index.js:/usr/src/index.js platziapp



Abrimos el navegador en *http://localhost:3200/*. En el archivo *index.js* realizamos nuevamente un cambio cosmetico. Y veras que en la terminal se detectaron los cambios y volvio a iniciar Node.

![](https://i.imgur.com/CEyblUp.png)

En realidad este enfoque no es muy practico porque solo se esta montando un archivo. Recuerda el instructor intento hacer el *bind mount* no solo al archvo *index.js* sino a todo el directorio, y lo que paso fue que *no pudo encontrar las dependencias*. ¿Extraño NO? 

Pues cuando le dimos la instruccion:

    COPY [".", "/usr/src/"]

Es que en realidad las dependencias no estan, porque nunca se hizo *RUN npm install* se esta creando un conflicto al montar todo el folder. Por eso toco hacerlo solo con un archivo.

#### RESUMEN: 

A la hora de instalar dependencias, debemos separar el copiado de los archivos de referencia de la instalación (como package.json o requirements.txt, los cuales irían antes de la ejecución de npm install o pip install) del copiado del resto de los archivos de la app (los cuales irían despues de la instalación de las dependencias).

De esta manera nos ahorramos la instalación de dependencias cada vez que cambiamos algo de la app, a excepción de cuando lo que cambian son los archivos de referencia.



## Docker Networking

La aplicacion funciona, pero no se puede conectar a Mongo, por lo que haremos el deber de solucionarlo.

Como esta configurado la aplicacion, asume Mongo debe estar instalado en el localhost.

![](https://i.imgur.com/1yqwwoE.png)

Sin embargo, una solucion podria ser tener otro contenor en Mongo, y de alguna manera que se conecten entre si

Entonces habria que crear otro contenedor, y conectarlo de alguna manera con el que ya esta funcionando. 

### Docker network

Observemos las redes virtuales por defecto:

    docker network ls

![](https://i.imgur.com/MoXIrkU.png)

Tenemos tres:

- bridge: compatibilidad con versiones anteriores
- host: una representacion real de la maquina host
- none: para que un contenedor no tenga ningun tipo de acceso a red, ni internet ni nada


#### Creando nuestra propia red

    docker network create --attachable platzinet

Con la opcion --attachable permitimos que los contenedores se conecten a ella. Si quisieramos mas informacion:

    docker network inspect platzinet 

### Creando y conectando los contenedores

El primero seria crear uno nuevo con la imagen de *mongo* que esta guardada, y verificar que este corriendo:

    docker run -d --name db mongo:5.0
    docker ps

Y conectarlo a la red *platzinet*, e inspeccionar dicha red:

    docker network connect platzinet db
    docker network inspect platzinet

![](https://i.imgur.com/ujtIKg4.png)

#### Contenedor de la aplicacion

Como ves en la linea de codigo que describimos al comienzo de esta seccion, busca la BD en el localhost, o a traves de la variable de entorno *MONGO_URL*.

Al momento de crear el contenedor le añadimos una descripcion a esa variable de entorno...Pero antes modifiquemos el dockerfile, porque ya no nos interesa la practica de la seccion anterior.

    CMD ["node", "index.js"]

Y el build:

    docker build -t platziapp .

Y ahora si, con la opcion *env* se asigna la variable de entorno *MONGO_URL* al parameto *mongodb*, y acontinuacion el *hostname* del contenedor donde va a estar mongo, que para este caso, seria el nombre del contenedor, seguido del puerto de mongo(27017), y el nombre de la BD(test):

    docker run -d --name app -p 3000:3000 --env MONGO_URL=mongodb://db:27017/test platziapp 
    
Solo hace falta agregar el container *app* a la red:

    docker network connect platzinet app

Si hacemos un inspect, igualmente encontraremos ya los dos contenedores en la red:

    docker network inspect platzinet

![](https://i.imgur.com/6QEwi4m.png)

#### Abriendo el Browser

![](https://i.imgur.com/ZkqFZLx.png)

Exito Rotundo!!!